# step05 make text samples

- Read step04 results (dates), make actual document samples

In [ ]:
import glob, os

def read_documents(path):
    
    file = open(path, "r")
    lines = file.readlines()
    file.close()
    
    date2doc = {}
    date = ""
    doctype = ""
    
    document = {}
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        
        if len(line)==0:
            if date in date2doc:
                continue
            else:
                date2doc[date] = document
                document = {}
                date = ""
                
        else:
            section = line.split("    || ")[0]
            content = line.split("    || ")[1]
        
            if "DATE"==section:
                date = content
            elif "doctype"==section:
                doctype = content

            else:
                key = doctype+"\t"+section
                if key in document:
                    content_tmp = document[key]
                    content_tmp.append(content)
                    document[key] = content_tmp
                else:
                    document[key] = [content]
                
                #print("document: ", document)
                
    if len(document)>0:
        if date not in date2doc:
            date2doc[date] = document
            document = {}
            date = ""
    
    return date2doc

def document_in_line(document):
    in_line = []
    for key in document:
        doc_type = key.split("\t")[0]
        section  = key.split("\t")[1]
        in_line.append(section)
        
        content = document[key]
        for c in range(len(content)):
            in_line.append(content[c])
            
    return " [SEP] ".join(in_line).strip()
        
    
def make_samples(filepath, mode):
    departments = glob.glob(filepath+"/"+str(mode)+"/*")
    departments.sort()

    for d in range(len(departments)):
        sample_count = 0
        
        department = departments[d].split("/")[-1].replace(".txt", "")
        print("department: ", department)

        file = open(departments[d], "r")
        lines = file.readlines()
        file.close()

        last_filepath = ""

        questions = []

        for l in range(1, len(lines)):
            line = lines[l].strip("\n")
            dates = []
            labels = []

            filepath = line.split("\t")[0]
            patient  = filepath.split("/")[-1].replace(".txt", "")
            dates  = [line.split("\t")[1], line.split("\t")[3], line.split("\t")[5], line.split("\t")[7], line.split("\t")[9]]
            labels = [line.split("\t")[2], line.split("\t")[4], line.split("\t")[6], line.split("\t")[8], line.split("\t")[10]]

            if last_filepath!=filepath:
                if len(questions)>0:
                    outpath = "./data/05_samples/"+str(mode)
                    if not os.path.exists(outpath):
                        os.makedirs(outpath)

                    file = open(outpath+"/"+str(department)+"_"+str(patient_last)+".txt", "w")
                    file.write("\n".join(questions))
                    file.close()
                    questions = []
                
                date2doc = read_documents(filepath)
                
                
            question = []
            question.append("[START_QUESTION]")
            for d in range(len(dates)):
                option = str(filepath)+"\t"+str(labels[d])+"\t"+str(dates[d])+"\t"+\
                          document_in_line(date2doc[dates[d]])
                question.append(option)
            question.append("[END_QUESTION]")

            
            questions.append("\n".join(question)+"\n")

            
            last_filepath = filepath
            patient_last = patient
            
            
                
        if len(questions)>0:
            outpath = "./data/05_samples/"+str(mode)
            if not os.path.exists(outpath):
                os.makedirs(outpath)

            file = open(outpath+"/"+str(department)+"_"+str(patient_last)+".txt", "w")
            file.write("\n".join(questions))
            file.close()
            questions = []

    
filepath = "./data/04_selected_idx"
mode = "train"
make_samples(filepath, mode)
print("Done training set")

filepath = "./data/04_selected_idx"
mode = "test"
make_samples(filepath, mode)
print("Done test set")
    